# 自动求导机制

该笔记对AutoGrad（自动求导）的原理和操作方法进行了全景式展示。尽管把这些都搞清楚没太大的必要，但是我们还是建议你能熟悉一下相关的概念，这有助于完成更加高效，清晰的代码，也有助于进行程序的调试。

## 从反向传播中剔除一部分子图

每一个Variable都有一个属性`requires_grad`，该属性可以让该节点从反向梯度计算中排除，进而提高运行效率。

如果输入中的任何一个节点需要计算梯度，那么输出节点也是需要计算梯度的。反之如果所有的输入节点都不需要计算梯度，输出节点才不需要计算梯度。如果一个子图中的所有节点都不需要梯度计算，那么该子图就不会执行反向传播。

Variable都是默认不需要计算梯度的。

In [2]:
import torch

x = torch.randn(5,5)
y = torch.randn(5,5)
z = torch.randn((5,5),requires_grad=True)
a = x+y
print(a.requires_grad)
b = a+z
print(b.requires_grad)

False
True


这一功能可以有效地冻结模型的部分参数，或者说你知道哪一些节点的梯度是不需要计算的。比如你想微调某一个预训练好的CNN模型的参数，那么可以将它的`requires_grad`属性关闭，关闭之后，这些节点的正向传播计算结果就不再缓存。当你需要微调某一层的参数时，你可以重建这一部分，然后使用优化器更新参数。

In [1]:
import torchvision
from torch import nn,optim

model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
    
model.fc = nn.Linear(512,100)
optimizer = optim.SGD(model.fc.parameters(),lr=1e-2,momentum=0.8)

## 自动求导机制如何对历史进行编码

自动求导是一种反向自动微分系统。从概念上讲，自动求导机制维护了一个记录该值是如何被计算出来的图，这张图是一个有向无环图，它的叶节点是输入Tensor而根节点是输出Tensor。从根节点向叶节点回溯，就可以利用链式法则来自动计算梯度。

在程序内部，自动求导机制实际上是一个由`Function`对象来构成的图，可以调用它的apply()魔术方法计算图的结果。当执行前向计算时，自动求导机制就同时执行所要求的计算，并构建一个图来表示计算梯度的函数（每一个torch.Tensor变量的grad_fn属性是该图的入口）。当前向计算结束时，就可以反向计算各个节点出的梯度。

有很重要的一点是，这个反向传播的图在每一次迭代中都会被重新构建，正是这一特点使得代码中可以使用任意的控制流语句，这些不同的控制语句可能会形成完全不同的反向传播计算图。在你开始训练之前，并不需要对反向传播的计算图做任何设计，每一个正向计算图都会自动生成相应的反向计算图。

## 自动求导机制中的原地变换操作

在自动求导机制中支持原地变换是一个很难的事情，在大多数情况下，我们并不鼓励这样操作。自动求导机制积极有效的存储释放和重用让它的效率很高，使得原地变换所能带来的存储节约非常有限。除非面临特别大的存储压力，否则不要使用任何原地变换。

不要使用原地变换的两个主要原因是：
* 原地变换会覆写某节点的值，而这个节点的值在计算梯度是必须使用
* 原地变换操作要求实现一个支持覆写的计算图，这是很困难的

## 原地变换正确性检查

每一个Tensor都有一个版本计数器，每一次它被变动时，版本计数器就会递增。如果有一个函数为了反向计算而保存了一个Tensor，那么对应的版本计数器也会被保存。一旦你访问saved_tensors方法，版本计数器就会检查一次，如果它要比存储的值大，那么就会报错。所以，如果你使用原地变换函数，也没有报错，那么反向计算梯度是没有问题的。